In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
aoms_2022 = pd.read_csv('./data/AOMS/2022/aoms_2022.csv',low_memory=False )

In [3]:
aoms_2023 = pd.read_csv('./data/AOMS/2023/aoms_2023.csv',low_memory=False )

In [4]:
aoms_2023.info()
aoms_2023.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9852716 entries, 0 to 9852715
Data columns (total 25 columns):
 #   Column             Dtype  
---  ------             -----  
 0   TM_STMP            object 
 1   MS_DATE            int64  
 2   MSG_TYPE           object 
 3   MS_IO              object 
 4   FLIGHT_NUM         object 
 5   MS_SID             int64  
 6   MS_AIRP            object 
 7   MS_AIRP_FN         object 
 8   MS_STIME           int64  
 9   MS_CTIME           int64  
 10  MS_LINE            object 
 11  MS_ATIME           float64
 12  MS_DELAY           int64  
 13  MS_CAUSE           object 
 14  MS_DECITY          float64
 15  MS_PAX             object 
 16  MG_REASON_AIRPORT  object 
 17  MS_CODE            object 
 18  MG_USECODE         int64  
 19  MS_PURPO           int64  
 20  REF1               object 
 21  REF2               float64
 22  REF3               float64
 23  REF4               float64
 24  REF5               float64
dtypes: float64(6), int

,TM_STMP,MS_DATE,MSG_TYPE,MS_IO,FLIGHT_NUM,MS_SID,MS_AIRP,MS_AIRP_FN,MS_STIME,MS_CTIME,...,MS_PAX,MG_REASON_AIRPORT,MS_CODE,MG_USECODE,MS_PURPO,REF1,REF2,REF3,REF4,REF5
0,23/01/02 00:32:18.120000000,20230108,I,A,DL7687,0,NGO,NAGOYA,1545,1545,...,Y,NaN,SL,0,0,DL,NaN,NaN,NaN,NaN
1,23/01/02 00:32:18.146000000,20230108,I,A,JL5219,0,NGO,NAGOYA,1545,1545,...,Y,NaN,SL,0,0,JL,NaN,NaN,NaN,NaN
2,23/01/02 00:32:18.169000000,20230108,I,A,KL4257,0,NGO,NAGOYA,1545,1545,...,Y,NaN,SL,0,0,KL,NaN,NaN,NaN,NaN
3,23/01/02 00:32:18.195000000,20230108,I,A,OM6717,0,NGO,NAGOYA,1545,1545,...,Y,NaN,SL,0,0,OM,NaN,NaN,NaN,NaN
4,23/01/02 00:32:18.225000000,20230108,I,A,DL7904,0,CTS,SAPPORO,1715,1715,...,Y,NaN,SL,0,0,DL,NaN,NaN,NaN,NaN


In [5]:

sorted_df = aoms_2023.sort_values(by='TM_STMP')
is_sorted = aoms_2023.equals(sorted_df)
if is_sorted:
    print("데이터프레임은 'TM_STMP' 열을 기준으로 정렬되어 있습니다.")
else:
    print("데이터프레임은 'TM_STMP' 열을 기준으로 정렬되어 있지 않습니다.")


데이터프레임은 'TM_STMP' 열을 기준으로 정렬되어 있지 않습니다.


In [6]:
# TM_STMP를 datetime으로 변환하고 TM_STMP으로 Sort
aoms_2023['TM_STMP'] = pd.to_datetime(aoms_2023['TM_STMP'], format='%y/%m/%d %H:%M:%S.%f')
aoms_2023 = aoms_2023.sort_values(['TM_STMP'])

In [7]:

pd.set_option('display.max_columns', None) 
# filtered = aoms_2023[(aoms_2023['MS_DATE'] == 20230107) & (aoms_2023['FLIGHT_NUM'] == 'TW175')]
filtered = aoms_2023[aoms_2023['FLIGHT_NUM'] == 'TW175']
# display(filtered)
filtered['MS_DATE'].unique() # 운항편명 Flight identifier이라는 건... (여러 스케쥴이 보이니까) 한번 뜨는 거에 대한 건 아닌듯
a = aoms_2023['FLIGHT_NUM'].unique()
np.set_printoptions(threshold=np.inf)
a
aoms_2023['MS_DECITY']

21554     NaN
21555     NaN
21556     NaN
21557     NaN
21558     NaN
           ..
9600848   NaN
9600849   NaN
9600850   NaN
9600851   NaN
9600852   NaN
Name: MS_DECITY, Length: 9852716, dtype: float64

In [8]:
#['TM_STMP']열 외에 다른 모든 값이 중복인 행들을 삭제
aoms_2023 = aoms_2023.drop_duplicates(subset=aoms_2023.columns.difference(['TM_STMP']))

In [9]:
# 
# df = df.groupby(['FLT_PK']).last()

aoms_2023 = aoms_2023.loc[aoms_2023['MS_LINE'] == 'D']    # 국내선만
aoms_2023 = aoms_2023.loc[aoms_2023['MS_AIRP'].notna()]   # 항공사 표기가 없는 경우 제외
aoms_2023 = aoms_2023.loc[aoms_2023['MSG_TYPE'] == 'U']    # D와 I 제거


aoms_2023 = aoms_2023.loc[(aoms_2023['MS_DELAY'] != 2)] # 결항 제거
aoms_2023 = aoms_2023.loc[(aoms_2023['MS_DELAY'] != 3)] # 회항 제거

# 항공 통계와 동일한 공항, 항공사로 제한
domestic = ['ICN', 'GMP', 'PUS', 'CJU', 'TAE', 'CJJ', 'KWJ', 'RSU', 'USN', 'WJU', 'HIN', 'KPO', 'YNY', 'KUV', 'MWX']
airline = ['OZ', '7C', 'LJ', 'KE', 'TW', 'BX', 'RS', 'RF', '4V', 'ZE']
aoms_2023 = aoms_2023.loc[(aoms_2023['MS_AIRP'].isin(domestic))] #MS_AIRP가 항공통계에 잡히는 공항인 것만

# FLIGHT 넘버에 앞 두자리- 항공통계 잡는 항공사만
aoms_2023['AIRLINE_PREFIX'] = aoms_2023['FLIGHT_NUM'].str[:2]
aoms_2023 = aoms_2023.loc[(aoms_2023['AIRLINE_PREFIX'].isin(airline))] 

# 9월의 경우만 확인해봄
a = aoms_2023.loc[aoms_2023['TM_STMP'].dt.month == 9]

In [10]:
a.groupby(['AIRLINE_PREFIX']).size().sort_values(ascending=False)

AIRLINE_PREFIX
KE    1550
7C     116
OZ      65
LJ      45
BX      15
ZE      13
RS      10
TW       6
dtype: int64

In [11]:
a.groupby(['MS_AIRP']).size().sort_values(ascending=False)

MS_AIRP
PUS    1238
TAE     267
GMP     222
CJU      44
ICN      43
HIN       6
dtype: int64

In [12]:
aoms_2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17409 entries, 20950 to 9698767
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   TM_STMP            17409 non-null  datetime64[ns]
 1   MS_DATE            17409 non-null  int64         
 2   MSG_TYPE           17409 non-null  object        
 3   MS_IO              17409 non-null  object        
 4   FLIGHT_NUM         17409 non-null  object        
 5   MS_SID             17409 non-null  int64         
 6   MS_AIRP            17409 non-null  object        
 7   MS_AIRP_FN         17409 non-null  object        
 8   MS_STIME           17409 non-null  int64         
 9   MS_CTIME           17409 non-null  int64         
 10  MS_LINE            17409 non-null  object        
 11  MS_ATIME           4245 non-null   float64       
 12  MS_DELAY           17409 non-null  int64         
 13  MS_CAUSE           16361 non-null  object        
 14  MS_DE

In [13]:
# aoms_2023.groupby(['MS_CAUSE']).size().sort_values(ascending=False)
aoms_2023.groupby(['MS_CAUSE']).size()

MS_CAUSE
0     13773
20        1
30        6
81        1
AD        6
      ...  
WR        1
WV       24
WW        2
WX       30
ZZ        1
Length: 63, dtype: int64

In [16]:
con_ect = [60, "WX", "NW", "AW", "WG", "WI", "WR"]
con_weather = [61, "WY", "WV", "WC", 62,"WS", "WN", 63, 64,"WU", "WH", 65,"WW", "WD", 66, 67, 68, 69,]
aoms_2023[aoms_2023['MS_CAUSE'].isin(con_weather) == True].groupby('MS_CAUSE').size()

MS_CAUSE
WC     7
WD    17
WN     3
WV    24
WW     2
dtype: int64

In [17]:
aoms_2023[aoms_2023['MS_CAUSE'].isin(con_ect) == True].groupby('MS_CAUSE').size()

MS_CAUSE
AW      2
NW    120
WG      5
WI      6
WR      1
WX     30
dtype: int64

In [19]:
aoms_2023.groupby('MS_DELAY').size()

MS_DELAY
0    14546
1     2863
dtype: int64